# Motion Detection Test

This is an implementation of the OpenCv tutorial found [here](https://docs.opencv.org/master/d1/dc5/tutorial_background_subtraction.html) on background subtraction.

Run to see results, hit escape to exit.

In [1]:
import cv2 as cv

ESCAPE_KEY = 27
KNN_DISTANCE_THRESHOLD = 1200
MINIMUM_BOUNDING_BOX_SIZE = 25

# using test video file
test_video = cv.VideoCapture('car-misses-exit.mp4')

# using Gaussian mixture background subtraction
background_subtraction = cv.createBackgroundSubtractorKNN(dist2Threshold=KNN_DISTANCE_THRESHOLD)

while True:
    
    # get the current frame
    ret, frame = test_video.read()
    
    # end of video
    if frame is None:
        break
    
    # get foreground mask
    foreground_mask = background_subtraction.apply(frame)
    
    # find contours and draw them
    contours, hierarchy = cv.findContours(foreground_mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(foreground_mask, contours, -1, (0,255,0), 3)
    
    # disregard children contours
    try: 
        hierarchy = hierarchy[0]
    except: 
        hierarchy = []

    # get dimensions for contour boxes
    height, width = foreground_mask.shape
    min_x, min_y = width, height
    max_x = max_y = 0

    # computes the bounding box for the contour
    for contour, hier in zip(contours, hierarchy):
        (x,y,w,h) = cv.boundingRect(contour)
        min_x, max_x = min(x, min_x), max(x+w, max_x)
        min_y, max_y = min(y, min_y), max(y+h, max_y)
        
        # draw the contour bounding box on the frame
        if w > MINIMUM_BOUNDING_BOX_SIZE and h > MINIMUM_BOUNDING_BOX_SIZE:
            cv.rectangle(frame, (x,y), (x+w,y+h), (255, 0, 0), 2)
        
    # display frame number in top left
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(test_video.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # display both frame and foreground mask
    cv.imshow('Frame', frame)
    cv.imshow('Foreground Mask', foreground_mask)
    
    # if escape key is pressed, exit
    keyboard = cv.waitKey(30)
    if keyboard == ESCAPE_KEY:
        cv.destroyAllWindows()
        break
        
cv.destroyAllWindows()